In [18]:
import cv2
import numpy as np

# Open the video capture
cap = cv2.VideoCapture("generated-video.mp4")
ret, frame = cap.read()  # Read the first frame
cap.release()

if ret:
    # frame shape: (height, width, 3) for RGB
    print("2D RGB vector shape:", frame.shape)
    print("2D RGB array (partial):")
    print(frame)  # This will print the full 2D array with RGB per pixel

    # To print a small region for clarity
    print("Sample (first 2 rows, 4 columns):")
    print(frame[:2, :4, :])  # Displays RGB for a 2x4 pixel region
else:
    print("Failed to read video.")

2D RGB vector shape: (704, 1248, 3)
2D RGB array (partial):
[[[164 113  89]
  [164 113  89]
  [164 113  89]
  ...
  [ 67  28  15]
  [ 67  28  15]
  [ 67  28  15]]

 [[164 113  89]
  [164 113  89]
  [164 113  89]
  ...
  [ 67  28  15]
  [ 67  28  15]
  [ 67  28  15]]

 [[164 113  89]
  [164 113  89]
  [164 113  89]
  ...
  [ 67  28  15]
  [ 67  28  15]
  [ 67  28  15]]

 ...

 [[ 30  31  82]
  [ 31  32  83]
  [ 30  33  81]
  ...
  [  5   2   0]
  [  4   1   0]
  [  4   1   0]]

 [[ 33  33  81]
  [ 34  34  82]
  [ 32  37  79]
  ...
  [  5   2   0]
  [  4   1   0]
  [  4   1   0]]

 [[ 31  31  79]
  [ 32  32  80]
  [ 30  34  76]
  ...
  [  5   2   0]
  [  4   1   0]
  [  4   1   0]]]
Sample (first 2 rows, 4 columns):
[[[164 113  89]
  [164 113  89]
  [164 113  89]
  [164 113  89]]

 [[164 113  89]
  [164 113  89]
  [164 113  89]
  [164 113  89]]]


In [15]:
# Find the color of the center pixel in the frame
height, width, _ = frame.shape
center_y = height // 2
center_x = width // 2
center_pixel = frame[center_y, center_x]
print(f"Center pixel coordinates: ({center_x}, {center_y})")
print(f"Center pixel color (RGB): {center_pixel}")

Center pixel coordinates: (624, 352)
Center pixel color (RGB): [ 86 223  66]


In [19]:
# Replace pixels close to color (86, 223, 66) with white
target_color = np.array([86, 223, 66])
tolerance = 15
lower = target_color - tolerance
upper = target_color + tolerance

# Create mask for target color range
mask = cv2.inRange(frame, lower, upper)

# Replace matching pixels with white
frame[mask > 0] = [255, 255, 255]

# Save the modified frame as an image
cv2.imwrite("frame_with_target_color_replaced.png", frame)
print("Saved as frame_with_target_color_replaced.png")

Saved as frame_with_target_color_replaced.png


In [21]:
# Find the largest rectangle of white pixels (255, 255, 255) in the frame
white_mask = np.all(frame == [255, 255, 255], axis=-1).astype(np.uint8)

# Use OpenCV to find contours of white regions
contours, _ = cv2.findContours(white_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

max_area = 0
largest_rect = None

for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    area = w * h
    if area > max_area:
        max_area = area
        largest_rect = (x, y, x + w - 1, y + h - 1)  # (x_min, y_min, x_max, y_max)

if largest_rect:
    x_min, y_min, x_max, y_max = largest_rect
    top_left = (x_min, y_min)
    top_right = (x_max, y_min)
    bottom_left = (x_min, y_max)
    bottom_right = (x_max, y_max)
    print(f"Largest white rectangle coordinates:")
    print(f"Top Left: {top_left}")
    print(f"Top Right: {top_right}")
    print(f"Bottom Left: {bottom_left}")
    print(f"Bottom Right: {bottom_right}")
    print(f"Area: {max_area}")
else:
    print("No white rectangle found.")

Largest white rectangle coordinates:
Top Left: (494, 236)
Top Right: (783, 236)
Bottom Left: (494, 424)
Bottom Right: (783, 424)
Area: 54810


In [23]:
# Replace all pixels within the largest white rectangle in every frame of the video with white pixels
import cv2
import numpy as np

# Open the video
cap = cv2.VideoCapture("generated-video.mp4")
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = None

largest_rect = (494, 236, 783, 424)

# Use the coordinates from previous cell (update these if needed)
x_min, y_min, x_max, y_max = largest_rect  # Make sure this is set from previous cell

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter("video_with_white_rectangle.mp4", fourcc, fps, (width, height))

for i in range(frame_count):
    ret, frame = cap.read()
    if not ret:
        break
    # Replace rectangle region with white
    frame[y_min:y_max+1, x_min:x_max+1] = [255, 255, 255]
    out.write(frame)

cap.release()
out.release()
print("Saved as video_with_white_rectangle.mp4")

Saved as video_with_white_rectangle.mp4


In [ ]:
# Apply an arbitrary image to the detected rectangle for every frame in the video
# This uses the helper in apply_image_to_video.py which resizes and pads the image
from video_processing import apply_image_to_video

video_path = 'generated-video.mp4'
image_path = 'creative-01.jpg'  # any image you want to place
rect = (494, 236, 783, 424)  # (x_min, y_min, x_max, y_max)
out_path = 'screensaver-01.mp4'

print('Starting to write', out_path)
apply_image_to_video(video_path, image_path, rect, out_path)
print('Finished writing', out_path)

Starting to write screensaver-01.mp4
Finished writing screensaver-01.mp4
Finished writing screensaver-01.mp4


In [ ]:
# Run per-frame detection and place an image into detected rectangles for each frame
from video_processing import apply_image_to_video_per_frame

video_path = 'input-video-03.mp4'
image_path = 'creative-02.png'
out_path = 'output-31.mp4'

print('Starting per-frame processing on', video_path)
apply_image_to_video_per_frame(video_path, image_path, (50,225,60), out_path, tolerance=20, min_area=1000)
print('Finished; wrote', out_path)

Starting per-frame processing on input-video-03.mp4


libpng warning: iCCP: known incorrect sRGB profile


Finished; wrote output-31.mp4
